In [1]:
# import dependencies for "poller.py"
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
import pandas as pd
import redis
import pickle

# import dependencies for "extensions.py"
import os
from dotenv import load_dotenv
from sklearn.utils._param_validation import InvalidParameterError
import zlib
import base64
from pymongo import MongoClient
from io import BytesIO
from scipy.sparse import save_npz, load_npz
import time


# import dependencies for "recommender_system.py"
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from pathlib import Path
from collections import Counter
import json
import nltk
import requests
from datetime import datetime, timedelta, tzinfo

# import dependencies for "elasticsearch_handle.py"
from elasticsearch import Elasticsearch
import json
import elastic_transport

# import dependencies for "elastic_exceptions.py"
from elastic_transport import TlsError, ConnectionTimeout, ConnectionError

In [2]:
# elasticsearch_handle
class InteractionNotFound(Exception):
    args = ("User doesn't have any interactions.", 110)


class ElasticConnectionError(TlsError):
    args = ("Elastic connection didn't established.", 120)


class ElasticsearchHandel:
    def __init__(self, elasticsearch_url, username, password, fingerprint):
        self.elasticsearch_url = elasticsearch_url
        self.username = username
        self.password = password
        self.fingerprint = fingerprint
        self.client = Elasticsearch(
            hosts=self.elasticsearch_url,
            basic_auth=(self.username, self.password),
            ssl_assert_fingerprint=self.fingerprint,
        )

    def get_index_v1(self, index_name, batch_size=100):
        setattr(self, index_name, [])
        index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        while True:
            query = {
                "query": {"match_all": {}},
                "size": batch_size,
                "from": from_index,
            }
            results = self.client.search(
                index=index_name,
                query={"match_all": {}},
                size=batch_size,
                from_=from_index,
            )
            instances = results["hits"]["hits"]

            all_instances.extend(instances)
            from_index += batch_size
            if len(instances) < 100:
                break

        setattr(self, index_name, [instance["_source"] for instance in all_instances])
        return getattr(self, index_name)

    def get_index(self, index_name, batch_size=100):
        setattr(self, index_name, [])

        # Create an Elasticsearch client
        # es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

        # Define a query that matches all documents
        query = {"query": {"match_all": {}}}

        # Use the count API to get the total count of documents
        result = self.client.count(index=index_name, body=query)

        # Access the total count
        total_count = result["count"]
        print(f"Total count of documents in index {index_name}: {total_count}")

        from_index = 0
        all_instances = []

        query = {
            "query": {"match_all": {}},
            "size": batch_size,
            "from": from_index,
        }
        results = self.client.search(
            index=index_name,
            query={"match_all": {}},
            size=total_count,
            from_=from_index,
        )
        instances = results["hits"]["hits"]

        all_instances.extend(instances)
        from_index += batch_size

        setattr(self, index_name, [instance["_source"] for instance in all_instances])
        return getattr(self, index_name)

    def get_interactions(self, index_name, user_id, batch_size=100):
        # setattr(self, index_name, [])
        # index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        query = {
            "match_phrase": {"userId": user_id},
        }

        results = self.client.search(
            index=index_name,
            query=query,
            size=batch_size,
            from_=from_index,
            timeout="1s",
        )
        # instances = results["hits"]["hits"][0]
        hits = results["hits"].get("hits")

        if not hits:
            # raise ValueError("User doesn't have any interactions.")
            raise InteractionNotFound()

        return hits[0].get("_source")

    def get_user_network_polls(self, user_id, batch_size=100):
        # setattr(self, index_name, [])
        # index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        query = {
            "query": {
                "bool": {
                    "filter": {"term": {"userPrivatePolls.keyword": user_id}},
                    "must": {
                        "nested": {
                            "path": "userFolllowingIds",
                            "query": {
                                "terms": {
                                    "userFolllowingIds.keyword": [
                                        "list",
                                        "of",
                                        "following",
                                        "user",
                                        "ids",
                                    ]
                                }
                            },
                        }
                    },
                }
            }
        }

        results = self.client.search(
            index="users",
            query=query,
            size=batch_size,
            from_=from_index,
            timeout="1s",
        )
        # instances = results["hits"]["hits"][0]
        hits = results["hits"].get("hits")

        if not hits:
            # raise ValueError("User doesn't have any interactions.")
            raise InteractionNotFound()

        return hits[0].get("_source")

    def get_trend_polls(self, polls, ret_list=True):
        # polls = getattr(self, "polls")
        # trend_polls = sorted(polls, key=lambda x: (-x["numberOfPollups"], -x["numberOfVotes"], -x["numberOfLike"]))
        trend_polls = sorted(
            polls,
            key=lambda x: (
                -x["numberOfVotes"],
                -x["numberOfLike"],
                # -x["numberOfPollUp"],
            ),
        )
        return trend_polls

        # recs = trend_polls["id"]

        # print("\n", filtered_trend_polls, "\n")
        # setattr(self, "trend_polls", trend_polls)

    def export_index_to_file(self, index, index_file_path):
        try:
            with open(index_file_path, "w") as output:
                # for instance in self.instances:
                #        json.dump(instance["_source"], output, indent=4)
                json.dump(index, output, indent=4)
        except Exception as exp:
            print("Export Error", exp)


def get_index(self, index_name, batch_size=100):
    setattr(self, index_name, [])
    index_list = getattr(self, index_name)
    from_index = 0
    all_instances = []

    while True:
        # query = {"query": {"match_all": {}}, "size": batch_size, "from": from_index}
        results = self.client.search(
            index=index_name,
            query={"match_all": {}},
            size=batch_size,
            from_=from_index,
        )
        instances = results["hits"]["hits"]

        all_instances.extend(instances)
        from_index += batch_size
        if len(instances) < 100:
            break

    setattr(self, index_name, [instance["_source"] for instance in all_instances])
    return getattr(self, index_name)


""" if __name__ == "__main__":
    elasticsearch_url = "https://159.203.183.251:9200"
    username = "pollett"
    password = "9r0&rJP@19GY"
    fingerprint = "CE:AA:F7:FF:04:C7:31:14:78:9C:62:D4:CE:98:F9:EF:56:DA:70:45:37:14:E3:F8:66:0A:25:ED:05:04:83:EC"

    elastic_handle = ElasticsearchHandel(
        elasticsearch_url, username, password, fingerprint
    )

    polls = elastic_handle.get_index("polls")
    elastic_handle.export_index_to_file(polls, "./data/elas_polls.json")

    interactions = elastic_handle.get_index("userpollinteractions")
    elastic_handle.export_index_to_file(interactions, "./data/elas_interactions.json")
    # print(polls) """

' if __name__ == "__main__":\n    elasticsearch_url = "https://159.203.183.251:9200"\n    username = "pollett"\n    password = "9r0&rJP@19GY"\n    fingerprint = "CE:AA:F7:FF:04:C7:31:14:78:9C:62:D4:CE:98:F9:EF:56:DA:70:45:37:14:E3:F8:66:0A:25:ED:05:04:83:EC"\n\n    elastic_handle = ElasticsearchHandel(\n        elasticsearch_url, username, password, fingerprint\n    )\n\n    polls = elastic_handle.get_index("polls")\n    elastic_handle.export_index_to_file(polls, "./data/elas_polls.json")\n\n    interactions = elastic_handle.get_index("userpollinteractions")\n    elastic_handle.export_index_to_file(interactions, "./data/elas_interactions.json")\n    # print(polls) '

In [3]:
# extensions
def create_redis_pool(host, port, db):
    return redis.ConnectionPool(host=host, port=port, db=db)


def create_elastic_connection(
    poller_elasticsearch_url, poller_username, poller_password, poller_fingerprint
):
    try:
        elasticsearch_url = os.environ.get(poller_elasticsearch_url)
        username = os.environ.get(poller_username)
        password = os.environ.get(poller_password)
        fingerprint = os.environ.get(poller_fingerprint)
        elastic_handle = ElasticsearchHandel(
            elasticsearch_url, username, password, fingerprint
        )
        if elasticsearch_url and username and password and fingerprint:
            print(
                "[2. Environment variables were read correctly through (enivronment variables).]"
            )
            return elastic_handle
    except ConnectionTimeout as e:
        load_dotenv()
        elasticsearch_url = os.getenv("POLLER_ELASTICSEARCH_URL")
        username = os.getenv("POLLER_USERNAME")
        password = os.getenv("POLLER_PASSWORD")
        fingerprint = os.getenv("POLLER_FINGERPRINT")

        try:
            elastic_handle = ElasticsearchHandel(
                elasticsearch_url, username, password, fingerprint
            )
            if elasticsearch_url and username and password and fingerprint:
                print(
                    "[2. Environment variables were read correctly through (getenv).]"
                )
            return elastic_handle
        except TypeError:
            print("[2. Failed to read environment variables.]")
            print(e)


def remove_duplicates(input_list):
    seen = set()
    result = []

    for item in input_list:
        if item not in seen:
            seen.add(item)
            result.append(item)

    return result


def get_entity(redis_client, entity_key, extend_expiration=600):
    # Get the entity from Redis
    entity = redis_client.get(entity_key)

    # If the entity exists, reset the expiration time (e.g., to 60 seconds)

    if entity:
        redis_client.expire(entity_key, extend_expiration)
        print(f"The data for {entity_key} exists in Redis.")
        return entity


def check_key_exists(redis_client, key):
    if not redis_client.exists(key):
        raise KeyError(f"The key '{key}' does not exist in Redis.")


def find_duplicates(lst):
    seen = set()
    duplicates = set()
    for item in lst:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return duplicates


def save_matrix_to_mongodb(
    polls_tf_idf_matrix,
    collection,
    user_id,
    polls_df,
    filtered_trend_polls_list,
    timer=False,
):
    # Save the sparse matrix to a BytesIO buffer
    buffer = BytesIO()

    start_time = time.time()
    save_npz(buffer, polls_tf_idf_matrix)
    elapse_npz_time = time.time() - start_time

    # Reset the buffer position to the beginning
    buffer.seek(0)

    # Read the buffer content into binary data
    start_time = time.time()
    binary_data = buffer.read()
    binary_data_time = time.time() - start_time

    # Compress the binary data
    start_time = time.time()
    compressed_data = zlib.compress(binary_data)
    compressed_data_time = time.time() - start_time

    # Encode the compressed data as base64 for BSON storage
    start_time = time.time()
    encoded_data = base64.b64encode(compressed_data).decode("utf-8")
    encoded_data_time = time.time() - start_time

    polls_dict = polls_df.to_dict(orient="records")

    # Insert the encoded data into MongoDB
    start_time = time.time()
    collection.insert_one(
        {
            "user_id": user_id,
            "polls_tf_idf_matrix": encoded_data,
            # "concatenated_df": polls_dict,
            "filtered_trend_polls_list": filtered_trend_polls_list,
        }
    )
    insert_one_time = time.time() - start_time
    if timer:
        print(f"Function 'save_npz' took {elapse_npz_time:.4f} seconds.")
        print(f"Function 'buffer.read' took {binary_data_time:.4f} seconds.")
        print(f"Function 'zlib.compress' took {compressed_data_time:.4f} seconds.")
        print(f"Function 'base64.b64encode' took {encoded_data_time:.4f} seconds.")
        print(f"Function 'insert_one_time' took {insert_one_time:.4f} seconds.")


def read_matrix_from_mongodb(collection, user_id):
    # Retrieve the document from MongoDB
    result = collection.find_one({"user_id": user_id})

    if result:
        # Decode the base64 data
        encoded_data = result.get("polls_tf_idf_matrix", "")
        compressed_data = base64.b64decode(encoded_data)

        # Decompress the data
        binary_data = zlib.decompress(compressed_data)

        # Create a BytesIO object from the binary data
        buffer = BytesIO(binary_data)

        # Load the sparse matrix from the BytesIO buffer
        polls_tf_idf_matrix = load_npz(buffer)

        # Create a DataFrame from the concatenated_df
        concatenated_df = pd.DataFrame(result.get("concatenated_df", []))

        # Get other values
        user_id = result.get("user_id", "")
        filtered_trend_polls_list = result.get("filtered_trend_polls_list", [])

        return {
            "user_id": user_id,
            "polls_tf_idf_matrix": polls_tf_idf_matrix,
            "concatenated_df": concatenated_df,
            "filtered_trend_polls_list": filtered_trend_polls_list,
        }
    else:
        return None


def save_matrix_to_mongodb_file(matrix, collection, user_id):
    # Save the sparse matrix to a compressed file
    save_npz("matrix.npz", matrix)

    # Read the compressed file into a binary stream
    with open("matrix.npz", "rb") as file:
        binary_data = file.read()

    # Compress the binary data
    compressed_data = zlib.compress(binary_data)

    # Encode the compressed data as base64 for BSON storage
    encoded_data = base64.b64encode(compressed_data).decode("utf-8")

    # Insert the encoded data into MongoDB
    collection.insert_one({"user_id": user_id, "sparse_matrix": encoded_data})

In [4]:
# recommender_system
nltk.download("punkt")
nltk.download("stopwords")
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

tf_idf = TfidfVectorizer(stop_words="english")


def encode_topics(df):
    # topics = df["topics"].str.get_dummies(sep=",")
    # topics = df["topics"].apply( topicfor topic in topics  )
    one_hot_encoded = (
        pd.get_dummies(df["topics"].apply(pd.Series).stack()).groupby(level=0).sum()
    )
    df = pd.concat([df, one_hot_encoded], axis=1)
    # print(df)
    return df


def set_index(df, index_column="poll_ID"):
    df.set_index(index_column, inplace=True)
    return df


def reset_index(df):
    df.reset_index()
    return df


def check_column_type(df, column_name, check_type):
    column_index = df.columns.get_loc(column_name)
    for i in range(len(df)):
        if not isinstance(df.iloc[i, column_index], check_type):
            print(
                f"error: {df.iloc[i, 0], df.iloc[i, 1],df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4]}"
            )


def preprocess_text(text):
    tokens = nltk.tokenize.word_tokenize(text)
    # tokens = [word.lower() for word in tokens if type(word) is str]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    stop_words = set(nltk.corpus.stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    processed_text = " ".join(tokens)

    return processed_text


def preprocess_list(field_list):
    ret_list = []
    stop_words = set(nltk.corpus.stopwords.words("english"))
    for item in field_list:
        tokens = nltk.tokenize.word_tokenize(item)
        # tokens = [word.lower() for word in tokens if type(word) is str]
        tokens = [word.lower() for word in tokens]
        tokens = [word for word in tokens if word not in string.punctuation]
        tokens = [word for word in tokens if word not in stop_words]
        processed_text = " ".join(tokens)
        ret_list.append(processed_text)

    return ret_list


def create_tf_idf_matrix(df, column):
    # print(f"{df[column]} is {df[column].dtype} and {df[column].dtype is list} {list}: ")
    df[column] = df[column].apply(lambda x: " ".join(x))
    df[column] = df[column].apply(preprocess_text)

    return tf_idf.fit_transform(df[column])


def create_souped_tf_idf_matrix(df):
    df["topics"] = df["topics"].apply(preprocess_list)
    df["question"] = df["question"].apply(preprocess_text)

    # Create a new soup feature
    df["soup"] = df.apply(create_soup, axis=1)

    return tf_idf.fit_transform(df["soup"])


def create_soup(df):
    res = (
        df["question"]
        + " "
        + " ".join(df["options"])
        + " "
        + (4 * (" " + " ".join(df["topics"])))
    )
    # print(f"-----------------------------------\n* Processing: [{ }]")
    return res


def calc_cosine_similarity_matrix(tf_idf_matrix_1, tf_idf_matrix_2):
    # if tf_idf_matrix_1 is not None and tf_idf_matrix_2 is not None:
    return cosine_similarity(tf_idf_matrix_1, tf_idf_matrix_2)


def id_to_index(df, search_id):
    result = df[df["id"] == str(search_id)].index.values[0]
    print(result)

    if len(result) > 0:
        return result
    else:
        return None


def id_to_index2(df, id):
    try:
        if any(df["id"] == str(id)):
            # df.to_csv("df.csv", index=False)
            # print(
            #    f"---------------\nFound {id} at {df[df['id'] == str(id)].index.values[0]}"
            # )
            # print(f"\nWhich is equal to:\n{df[df['id'] == str(id)]}")
            return df[df["id"] == str(id)].index.values[0]

    except IndexError as e:
        print(f"erorrrrrrrrrrrrr:")
        print(f"{str(id)}")
        print(f"{df['id']==str(id)}")


def title_from_idx(df, idx):
    return df[df.index == idx]


def gen_recommendations(
    index,
    df,
    cosine_similarity_matrix,
    number_of_recommendations,
):
    # index = idx_from_title(df, original_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(
        similarity_scores, key=lambda x: x[1], reverse=True
    )

    recommendations_indices = [
        t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    ]
    recommendations = list(df["title"].iloc[recommendations_indices])
    # print(recommendations)
    # print(similarity_scores_sorted, type(similarity_scores_sorted))
    # recommendations_indices = [
    #    t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # recommendations_scores = [
    #    t[1] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # return (df["title"].iloc[recommendations_indices], recommendations_scores)

    return recommendations


def gen_rec_from_list_of_polls(
    interacted_polls,
    filtered_polls_df,
    cosine_similarity_matrix,
    number_of_recommendations,
):
    recommendations = []
    for poll_id in interacted_polls:
        index = id_to_index2(filtered_polls_df, poll_id)
        if index is not None:
            similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
            similarity_scores_sorted = sorted(
                similarity_scores, key=lambda x: x[1], reverse=True
            )

            recommendations_indices = [
                t[0]
                for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
            ]
            recs = list(filtered_polls_df["id"].iloc[recommendations_indices])

            # Filter out polls that have already been interacted with
            filtered_recs = [poll for poll in recs if poll not in interacted_polls]

            recommendations.append(filtered_recs)

        else:
            pass

        # index = id_to_index(polls, poll_id)
        # print(f"cosine_similarity_matrix:{len(cosine_similarity_matrix)}")
        # print(f"index:{index} | id:{poll_id}")

    flattened_recommendations = [
        item for sublist in recommendations for item in sublist
    ]
    flattened_recommendations = Counter(flattened_recommendations)
    n_most_recommended = flattened_recommendations.most_common(
        number_of_recommendations
    )
    n_most_recommended = [t[0] for t in n_most_recommended]
    # print(n_most_recommended)

    return n_most_recommended


def gen_rec_from_list_of_polls_df(
    interacted_polls,
    filtered_polls_df,
    cosine_similarity_matrix,
    number_of_recommendations,
):
    recommendations = []
    for poll_id in interacted_polls:
        index = id_to_index2(filtered_polls_df, poll_id)
        if index is not None:
            similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
            similarity_scores_sorted = sorted(
                similarity_scores, key=lambda x: x[1], reverse=True
            )

            recommendations_indices = [
                t[0]
                for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
            ]
            recs = list(filtered_polls_df["id"].iloc[recommendations_indices])

            # Filter out polls that have already been interacted with
            filtered_recs = [poll for poll in recs if poll not in interacted_polls]

            recommendations.append(filtered_recs)

        else:
            pass

        # index = id_to_index(polls, poll_id)
        # print(f"cosine_similarity_matrix:{len(cosine_similarity_matrix)}")
        # print(f"index:{index} | id:{poll_id}")

    flattened_recommendations = [
        item for sublist in recommendations for item in sublist
    ]
    flattened_recommendations = Counter(flattened_recommendations)
    n_most_recommended = flattened_recommendations.most_common(
        number_of_recommendations
    )
    n_most_recommended = [t[0] for t in n_most_recommended]

    filtered_df = filtered_polls_df[filtered_polls_df["id"].isin(n_most_recommended)]

    order_dict = {id: idx for idx, id in enumerate(n_most_recommended)}

    # Sort the filtered DataFrame based on the order
    # filtered_df["order"] = filtered_df["id"].map(order_dict)
    filtered_df.loc[:, "order"] = filtered_df.loc[:, "id"].map(order_dict)

    filtered_df = filtered_df.sort_values("order")

    # Drop the 'order' column if not needed
    filtered_df = filtered_df.drop(columns=["order"])

    # Reset the index if needed
    filtered_df = filtered_df.reset_index(drop=True)

    return filtered_df


def is_valid_limitations(limitations):
    if isinstance(limitations, dict):
        return (
            "allowedLocations" in limitations
            and "allowedGender" in limitations
            and "allowedAgeRange" in limitations
        )
    return False


def is_within_x_days_liifetime(timestamp):
    try:
        # Convert the timestamp to a datetime object
        time = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")

        # Calculate the time difference
        time_difference = datetime.now() - time

        return True if time_difference <= timedelta(days=10) else False
    except ValueError:
        # If the timestamp doesn't match the expected format, return False
        return False


def filter_timestamp(timestamp):
    try:
        # Convert the timestamp to a datetime object
        time = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")

        # Calculate the time difference
        time_difference = datetime.now() - time

        return time_difference
    except ValueError:
        # If the timestamp doesn't match the expected format, return None
        return None


def split_by_days(polls_df, days=10):
    filtered_df = polls_df[polls_df["createdAt"].apply(filter_timestamp).notna()]

    older_than_x_days = filtered_df[
        filtered_df["createdAt"].apply(filter_timestamp) >= timedelta(days=days)
    ]
    newer_than_x_days = filtered_df[
        filtered_df["createdAt"].apply(filter_timestamp) < timedelta(days=days)
    ]

    # Reset the index if needed
    older_than_x_days = older_than_x_days.reset_index(drop=True)
    newer_than_x_days = newer_than_x_days.reset_index(drop=True)

    return older_than_x_days, newer_than_x_days


def has_valid_date(date_str):
    # Convert the date string to a datetime object
    # date = pd.to_datetime(date_str)
    date = pd.to_datetime(date_str, utc=True).replace(tzinfo=None)

    # Get the current timestamp as a datetime object
    current_time = datetime.now()

    # Compare the date with the current timestamp
    return date > current_time


def remove_duplicates(input_list):
    seen = set()
    result = []

    for item in input_list:
        if item not in seen:
            seen.add(item)
            result.append(item)

    return result


def validate_polls_v1(polls_df, df_name, verbose=True):
    valid_polls = polls_df.loc[
        polls_df["valid"] & polls_df["endedAt"].apply(has_valid_date)
    ]

    invalid_polls = polls_df[
        ~polls_df["valid"] | ~polls_df["endedAt"].apply(has_valid_date)
    ]

    if verbose:
        print(f"valid_{df_name}_polls: {len(valid_polls)}")
        print(f"expired_{df_name}_polls: {len(invalid_polls)}")

    return valid_polls, invalid_polls
    return pd.concat([valid_polls, invalid_polls], ignore_index=False)


def validate_polls(polls_df, df_name, verbose=True):
    valid_polls = polls_df.loc[
        polls_df["valid"] & polls_df["endedAt"].apply(has_valid_date)
    ]

    invalid_polls = polls_df[~polls_df["id"].isin(valid_polls["id"].tolist())]
    # invalid_polls = polls_df.loc[
    #    ~polls_df["valid"] | ~polls_df["endedAt"].apply(has_valid_date)
    # ]

    if verbose:
        print(f"valid_{df_name}_polls: {len(valid_polls)}")
        print(f"expired_{df_name}_polls: {len(invalid_polls)}")

    return valid_polls, invalid_polls


def order_v1(
    recommended_polls_df=None,
    trend_polls_df=None,
    live_polls_flag=0,
    verbose=True,
):
    if not live_polls_flag:
        if recommended_polls_df is None:
            valid_trend_polls, invalid_trend_polls = validate_polls_v1(
                trend_polls_df, "trend", verbose
            )
            recommended_polls_df = pd.concat(
                [
                    valid_trend_polls,
                    invalid_trend_polls,
                ],
                ignore_index=False,
            )

        elif trend_polls_df is None:
            valid_recommended_polls, invalid_recommended_polls = validate_polls_v1(
                recommended_polls_df, "recommended", verbose
            )
            recommended_polls_df = pd.concat(
                [
                    valid_recommended_polls,
                    invalid_recommended_polls,
                ],
                ignore_index=False,
            )

        elif trend_polls_df is not None and recommended_polls_df is not None:
            valid_recommended_polls, invalid_recommended_polls = validate_polls_v1(
                recommended_polls_df, "recommended", verbose
            )
            valid_trend_polls, invalid_trend_polls = validate_polls_v1(
                trend_polls_df, "trend", verbose
            )

            recommended_polls_df = pd.concat(
                [
                    valid_recommended_polls,
                    valid_trend_polls,
                    invalid_recommended_polls,
                    invalid_trend_polls,
                ],
                ignore_index=False,
            )

        recommended_polls_df = recommended_polls_df.reset_index(drop=True)
        recommended_polls_df = recommended_polls_df["id"].tolist()
        recommended_polls_df = remove_duplicates(recommended_polls_df)

        return recommended_polls_df

    else:
        if recommended_polls_df is None:
            recommended_polls_df, _ = validate_polls_v1(
                trend_polls_df, "trend", verbose
            )

        elif trend_polls_df is None:
            recommended_polls_df, _ = validate_polls_v1(
                recommended_polls_df, "recommended", verbose
            )

        elif trend_polls_df is not None and recommended_polls_df is not None:
            valid_recommended_polls, invalid_recommended_polls = validate_polls_v1(
                recommended_polls_df, "recommended", verbose
            )
            valid_trend_polls, invalid_trend_polls = validate_polls_v1(
                trend_polls_df, "trend", verbose
            )

            recommended_polls_df = pd.concat(
                [
                    valid_recommended_polls,
                    valid_trend_polls,
                ],
                ignore_index=False,
            )

        recommended_polls_df = recommended_polls_df.reset_index(drop=True)
        recommended_polls_df = recommended_polls_df["id"].tolist()
        recommended_polls_df = remove_duplicates(recommended_polls_df)

        return recommended_polls_df


def order_v2(
    recommended_polls_df=None,
    trend_polls_df=None,
    live_polls_flag=False,
    verbose=True,
):
    if live_polls_flag:
        if recommended_polls_df is None:
            recommended_polls_df = validate_and_concat(trend_polls_df, "trend", verbose)

        elif trend_polls_df is not None:
            valid_recommended_polls = validate_and_concat(
                recommended_polls_df, "recommended", verbose
            )
            valid_trend_polls = validate_and_concat(trend_polls_df, "trend", verbose)

            recommended_polls_df = pd.concat(
                [valid_recommended_polls, valid_trend_polls], ignore_index=False
            )

        recommended_polls_df = recommended_polls_df.reset_index(drop=True)
        recommended_polls_df = recommended_polls_df["id"].tolist()
        recommended_polls_df = remove_duplicates(recommended_polls_df)

        return recommended_polls_df

    else:
        if recommended_polls_df is None:
            recommended_polls_df = validate_and_concat(trend_polls_df, "trend", verbose)

        elif trend_polls_df is None:
            recommended_polls_df = validate_and_concat(
                recommended_polls_df, "recommended", verbose
            )

        else:
            valid_recommended_polls = validate_and_concat(
                recommended_polls_df, "recommended", verbose
            )
            valid_trend_polls = validate_and_concat(trend_polls_df, "trend", verbose)

            recommended_polls_df = pd.concat(
                [valid_recommended_polls, valid_trend_polls], ignore_index=False
            )

        recommended_polls_df = recommended_polls_df.reset_index(drop=True)
        recommended_polls_df = recommended_polls_df["id"].tolist()
        recommended_polls_df = remove_duplicates(recommended_polls_df)

        return recommended_polls_df


def order_v3(
    recommended_polls_df=None,
    trend_polls_df=None,
    live_polls_flag=0,
    verbose=True,
):
    val_recommended_polls_df, inval_recommended_polls_df = validate_polls(
        recommended_polls_df, "recommended", verbose
    )

    val_trend_polls_df, inval_trend_polls_df = validate_polls(
        trend_polls_df, "trend", verbose
    )

    if live_polls_flag:
        recommended_polls_df = pd.concat(
            [
                val_recommended_polls_df,
                val_trend_polls_df,
            ],
            ignore_index=False,
        )
    else:
        recommended_polls_df = pd.concat(
            [
                val_recommended_polls_df,
                val_trend_polls_df,
                inval_recommended_polls_df,
                inval_trend_polls_df,
            ],
            ignore_index=False,
        )

    recommended_polls_df = recommended_polls_df.reset_index(drop=True)
    recommended_polls_df = recommended_polls_df["id"].tolist()
    recommended_polls_df = remove_duplicates(recommended_polls_df)

    return recommended_polls_df


def validate_polls_v3(polls_df, df_name, verbose=True):
    # Convert the 'timestamp_column' to a pandas datetime object
    # polls_df["liive"] = pd.to_datetime(
    #    polls_df["endedAt"], format="%Y-%m-%dT%H:%M:%S.%fZ", utc=True
    # )

    # polls_df["endedAt"] = pd.to_datetime(polls_df["endedAt"], utc=True)

    # Create a boolean mask based on whether the timestamp has passed
    # mask = (polls_df["endedAt"] < current_timestamp) & polls_df["valid"]

    # Condition 2: Check if the time in "endedAt" column has not passed
    current_timestamp = pd.Timestamp.utcnow()

    mask = (
        # (polls_df["endedAt"].notna())
        # (polls_df["liive"] < current_timestamp)
        (
            pd.to_datetime(
                polls_df["endedAt"],
                format="ISO8601",
                utc=True,
            )
            >= current_timestamp
        )
        & polls_df["valid"]
    )

    # Apply the mask to the DataFrame
    valid_polls = polls_df[mask]
    invalid_polls = polls_df[~mask]

    if verbose:
        print(f"valid_{df_name}_polls: {len(valid_polls)}")
        print(f"expired_{df_name}_polls: {len(invalid_polls)}")

    return valid_polls, invalid_polls


def order_v4(
    recommended_polls_df=None,
    trend_polls_df=None,
    live_polls_flag=0,
    verbose=True,
):
    val_recommended_polls_df, inval_recommended_polls_df = validate_polls_v3(
        recommended_polls_df, "recommended", verbose
    )

    val_trend_polls_df, inval_trend_polls_df = validate_polls_v3(
        trend_polls_df, "trend", verbose
    )

    if live_polls_flag:
        recommended_polls_df = pd.concat(
            [
                val_recommended_polls_df,
                val_trend_polls_df,
            ],
            ignore_index=True,
        )
    else:
        recommended_polls_df = pd.concat(
            [
                val_recommended_polls_df,
                val_trend_polls_df,
                inval_recommended_polls_df,
                inval_trend_polls_df,
            ],
            ignore_index=True,
        )

    recommended_polls_df = recommended_polls_df.reset_index(drop=True)
    recommended_polls_df = recommended_polls_df["id"].tolist()
    recommended_polls_df = remove_duplicates(recommended_polls_df)

    return recommended_polls_df


def list_to_df(polls_list, polls_df):
    # Filter the DataFrame based on the id_list
    filtered_df = polls_df[polls_df["id"].isin(polls_list)]

    # Create a dictionary to preserve the order
    order_dict = {id: idx for idx, id in enumerate(polls_list)}

    # Sort the filtered DataFrame based on the order
    filtered_df["order"] = filtered_df["id"].map(order_dict)
    filtered_df = filtered_df.sort_values("order")

    # Drop the 'order' column if not needed
    filtered_df = filtered_df.drop(columns=["order"])

    # Reset the index if needed
    filtered_df = filtered_df.reset_index(drop=True)
    return filtered_df


def filter_polls(row, user_limitations):
    if (
        row["pollType"] == "Public"
        and isinstance(row.get("pollLimitations"), dict)
        and all(k in user_limitations for k in ["Location", "Gender", "Age"])
        # and is_within_x_days_liifetime(row["createdAt"])
    ):
        user_location = user_limitations.get("Location")

        allowed_locations = row.get("pollLimitations").get("allowedLocations")
        if len(allowed_locations) == 0 or any(
            user_location == loc for loc in allowed_locations
        ):
            allowed_gender = row["pollLimitations"]["allowedGender"]
            user_gender = user_limitations["Gender"]
            if allowed_gender == "All" or allowed_gender == user_gender:
                allowed_age_range = row["pollLimitations"]["allowedAgeRange"]
                user_age = user_limitations["Age"]
                if (
                    allowed_age_range["minimumAge"]
                    <= user_age
                    <= allowed_age_range["maximumAge"]
                ):
                    return True

    return False


def get_allowed_private_polls(
    params,
    # url="https://dev.pollett.io/api/Recommend/Polls/GetPrivatePollThatUserCanSee",
    url,
):
    # API URL
    # url = "https://dev.pollett.io/api/Recommend/Polls/GetPrivatePollThatUserCanSee"

    # Parameters
    # params = {"userId": "bbe64b34-ba34-4fbd-a62f-e6c84c0423b4"}

    # Send a GET request to the API
    response = requests.get(url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        # allowed_polls_list = response.json().get("data")
        return response.json().get("data")
        # Process the data as needed

    else:
        # Handle the error
        print(f"Request failed with status code {response.status_code}")
        print(response.text)


# if __name__ == "__main__":
#    pd.set_option("display.max_colwidth", None)
#    pd.set_option("display.max_columns", None)
#
#    path = Path(__file__).parent.parent.parent.resolve()
#    path = str(path) + "/data/elas_polls.json"
#    polls_list = []
#    # polls = pd.read_json(str(path) + "/data/elas_polls.json")
#    with open(path, "r") as infile:
#        polls = json.load(infile)
#        for poll in polls:
#            # poll = poll["_source"]
#            # print(f"poll:\n{poll}")
#            polls_list.append(poll)
#
#    polls = pd.DataFrame.from_records(polls_list)
#
#    polls = encode_topics(polls)
#    print(polls)
#    # check_column_type(polls, 4, str)
#    tf_idf_matrix = create_tf_idf_matrix(polls, "question")
#    cosine_similarity_matrix = calc_cosine_similarity_matrix(
#        tf_idf_matrix, tf_idf_matrix
#    )

[nltk_data] Downloading package punkt to /home/eyz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eyz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# save this attribute for each user to a database
# useful_recommendation_ratio = interacted_rp / num_of_represented_rp

In [ ]:
# test current algorithm's functionality and quality
# mean_rate = all_useful_recommendation_rate / num_of_rates